In [18]:
import os
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Set up transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the full dataset
full_dataset = datasets.ImageFolder(root="data/train", transform=transform)

# Create indices for training and testing split
dataset_size = len(full_dataset)
indices = list(range(dataset_size))
train_indices, test_indices = train_test_split(indices, test_size=0.25, stratify=[full_dataset.targets[i] for i in indices])

# Create subsets
train_dataset = Subset(full_dataset, train_indices)
test_dataset = Subset(full_dataset, test_indices)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Checking the classes
print("Classes:", full_dataset.classes)


Classes: ['Blackheads', 'Cystic', 'Papules', 'Pustules', 'Rosacea', 'Whiteheads']


In [19]:
import torch
import torch.nn as nn
from torchvision import models

# Set the number of classes based on your dataset
num_classes = len(train_loader.dataset.dataset.classes)

# Load MobileNetV2 with pretrained weights
model = models.mobilenet_v2(pretrained=True)

# Replace the final layer to match the number of classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Move the model to the available device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


c:\Users\Nathan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Nathan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
from sklearn.metrics import confusion_matrix
import numpy as np

def evaluate_per_class(model, test_loader):
    model.eval()  # Set to evaluation mode
    all_preds = []
    all_labels = []

    # Initialize a confusion matrix
    conf_matrix = np.zeros((len(test_loader.dataset.dataset.classes), len(test_loader.dataset.dataset.classes)))

    with torch.no_grad():  # No need to compute gradients
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Send to device
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            # Update confusion matrix
            for t, p in zip(labels.view(-1), preds.view(-1)):
                conf_matrix[t.item(), p.item()] += 1

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate accuracy for each class
    class_accuracies = []
    for i in range(len(test_loader.dataset.dataset.classes)):
        class_correct = conf_matrix[i, i]
        class_total = conf_matrix[i].sum()
        class_accuracy = class_correct / class_total if class_total > 0 else 0
        class_accuracies.append(class_accuracy)
        print(f"Accuracy for class {test_loader.dataset.dataset.classes[i]}: {class_accuracy * 100:.2f}%")
    
    # Calculate overall accuracy
    accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    return accuracy, class_accuracies


In [21]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 5 # You can adjust this based on your dataset size and computational power

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/6, Loss: 0.3012
Epoch 2/6, Loss: 0.2778
Epoch 3/6, Loss: 0.2521
Epoch 4/6, Loss: 0.1383
Epoch 5/6, Loss: 0.0983
Epoch 6/6, Loss: 0.1136


In [27]:
# Get the class-wise accuracy and overall accuracy
overall_accuracy, class_accuracies = evaluate_per_class(model, test_loader)
print(f"Overall Test Accuracy: {overall_accuracy * 100:.2f}%")


Accuracy for class Blackheads: 55.00%
Accuracy for class Cystic: 29.17%
Accuracy for class Papules: 89.19%
Accuracy for class Pustules: 50.00%
Accuracy for class Rosacea: 88.00%
Accuracy for class Whiteheads: 68.42%
Overall Test Accuracy: 69.64%
